## Custom API Notebook

### Initialize the Node

In [ ]:
# stdlib
from typing import Any

# syft absolute
import syft as sy
from syft import SyftError
from syft import SyftSuccess

node = sy.orchestra.launch(name="test-domain-1", port="auto", dev_mode=True, reset=True)
domain_client = node.login(email="info@openmined.org", password="changethis")
domain_client.register(
    email="user@openmined.org",
    password="verysecurepassword",
    password_verify="verysecurepassword",
    name="New User",
)
domain_guest = node.login(email="user@openmined.org", password="verysecurepassword")

### Create a public custom API Endpoint by using the decorator

This allows node admin to create a new public endpoint by using only the decorator.

In [ ]:
@sy.api_endpoint(
    path="first.query",
    settings={"key": "value"},
)
def public_endpoint_method(
    context,
    query: str,
) -> Any:
    return context.settings["key"] == "value"


# Add it to the node.
response = domain_client.api.services.api.add(endpoint=public_endpoint_method)
response

In [ ]:
assert isinstance(response, SyftSuccess)

In [ ]:
assert len(domain_client.api.services.api.api_endpoints()) == 1

In [ ]:
# Once api refresh is done, remove this cell
domain_client = node.login(email="info@openmined.org", password="changethis")
domain_guest = node.login(email="user@openmined.org", password="verysecurepassword")

In [ ]:
assert domain_client.api.services.first.query(query="SELECT *")
assert domain_guest.api.services.first.query(query="SELECT *")

### Create public/private Custom API Endpoint using TwinAPIEndpoint

This allows the admin to create a public/private endpoint interface where the users can iteract with.

In [ ]:
@sy.mock_api_endpoint(settings={"Hello": "Public"})
def public_function(
    context,
) -> str:
    return "Public Function Execution"


@sy.private_api_endpoint(settings={"Hello": "Private"})
def private_function(
    context,
) -> str:
    return "Private Function Execution"


new_endpoint = sy.TwinAPIEndpoint(
    path="third.query",
    mock_function=public_function,
    private_function=private_function,
    description="Lore ipsulum ...",
)

# # Add it to the node.
response = domain_client.api.services.api.add(endpoint=new_endpoint)

In [ ]:
domain_client.api.services.api.api_endpoints()

In [ ]:
assert isinstance(response, SyftSuccess)
assert len(domain_client.api.services.api.api_endpoints()) == 2

In [ ]:
# Once api refresh is done, remove this cell
domain_client = node.login(email="info@openmined.org", password="changethis")
domain_guest = node.login(email="user@openmined.org", password="verysecurepassword")

In [ ]:
assert domain_client.api.services.third.query() == "Private Function Execution"

In [ ]:
assert domain_guest.api.services.third.query() == "Public Function Execution"

In [ ]:
domain_guest.api.services.third.query()

In [ ]:
@sy.syft_function_single_use(
    endpoint=domain_guest.api.services.third.query,
)
def job_function(endpoint):
    return endpoint()


# Create a new project
new_project = sy.Project(
    name="My Cool UN Project",
    description="Hi, I want to calculate the trade volume in million's with my cool code.",
    members=[domain_guest],
)

result = new_project.create_code_request(job_function, domain_guest)
assert isinstance(result, SyftSuccess)

In [ ]:
domain_client.requests[-1].approve()

In [ ]:
result = domain_guest.code.job_function(endpoint=domain_client.api.services.third.query)
result

In [ ]:
result = domain_guest.code.job_function(endpoint=domain_client.api.services.third.query)
result

In [ ]:
domain_client.api.services.third.query

In [ ]:
result = domain_guest.code.job_function(endpoint=domain_client.api.services.third.query)
result

In [ ]:
print(result)
if isinstance(result, SyftError):
    assert result.message == ""

In [ ]:
assert result.get() == "Private Function Execution"

In [ ]:
assert isinstance(domain_guest.api.services.third.query.private(), SyftError)

In [ ]:
result = domain_client.api.services.api.delete(endpoint_path="third.query")
assert isinstance(result, SyftSuccess)

In [ ]:
assert len(domain_client.api.services.api.api_endpoints()) == 1

## Updating Endpoints

First we'll create a new endpoint

In [ ]:
@sy.api_endpoint(
    path="test.update",
    settings={"key": "value"},
)
def new_public_function(
    context,
    query: str,
) -> Any:
    return context.settings["key"] == "value"


# Add it to the node.
response = domain_client.api.services.api.add(endpoint=new_public_function)
assert isinstance(response, SyftSuccess)
response

#### Update the public function

In [ ]:
@sy.mock_api_endpoint(settings={"Hello": "Public"})
def updated_public_function(
    context,
) -> str:
    return "Updated Public Function Execution"


response = domain_client.api.services.api.update(
    endpoint_path="test.update", mock_function=updated_public_function
)
assert isinstance(response, SyftSuccess)
response

#### Update the private function

In [ ]:
@sy.private_api_endpoint(settings={"Hello": "Private"})
def updated_private_function(
    context,
) -> str:
    return "Updated Private Function Execution"


response = domain_client.api.services.api.update(
    endpoint_path="test.update", private_function=updated_private_function
)
assert isinstance(response, SyftSuccess)
response

#### Update both functions with a pair that has a new signature

In [ ]:
@sy.mock_api_endpoint(settings={"Hello": "Public"})
def new_sig_public_function(context, new_parameter) -> str:
    return "Updated Public Function Execution"


@sy.private_api_endpoint(settings={"Hello": "Private"})
def new_sig_private_function(context, new_parameter) -> str:
    return "Updated Private Function Execution"


response = domain_client.api.services.api.update(
    endpoint_path="test.update",
    mock_function=new_sig_public_function,
    private_function=new_sig_private_function,
)
assert isinstance(response, SyftSuccess)
response

### Invalid update attempts
- Both functions empty
- Signature mismatch
- Non existing endpoint

#### Both functions are empty

In [ ]:
response = domain_client.api.services.api.update(endpoint_path="test.update")
assert isinstance(response, SyftError)

#### Signature mismatch

In [ ]:
@sy.mock_api_endpoint(settings={"Hello": "Public"})
def bad_public_function(context, foo) -> str:
    return "Updated Public Function Execution"


response = domain_client.api.services.api.update(
    endpoint_path="test.update", mock_function=bad_public_function
)
assert isinstance(response, SyftError)

Non Existing endpoint

In [ ]:
response = domain_client.api.services.api.update(
    endpoint_path="nonexistent", mock_function=bad_public_function
)
assert isinstance(response, SyftError)